In [1]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class objs():
    def __init__(self, x = 250, y = 250, max_bare = 60, size = 10, max_age = 300, moving = 10):
        self.x, self.y = x, y
        self.size = size
        self.bare = max_bare
        self.max_bare = max_bare
        self.age = max_age
        self.max_age = max_age
        self.moving = moving

class predator(objs):
    def __init__(self, x = 250, y = 250, max_bare = 100, size = 25, max_age = 500, moving = 1, feeding = 100):
        super().__init__(x, y, max_bare, size, max_age, moving)
        self.feeding = feeding

In [3]:
t = 0
boundary = 2000

In [4]:
As = [objs(x = int(np.random.random()*boundary), y = int(np.random.random()*boundary), moving = 10+10*(np.random.random()-.5)) for i in range(100)]

In [5]:
Bs = [predator(x = int(np.random.random()*boundary), y = int(np.random.random()*boundary), moving = 4) for i in range(5)]

In [6]:
%matplotlib qt

In [7]:
fig, ax = plt.subplots(figsize = (10, 10))
while True:
    for A in As:
        if A.age < 0:
            As.remove(A)
        else:
            # A.age -= 1
            
            if A.bare < 0:
                if len(As) < 1000:
                    A.bare = A.max_bare
                    As.append(objs(x = A.x, y = A.y, max_bare = (1+np.random.normal(0, .09))*A.max_bare, 
                                   size = (1+np.random.normal(0, .09))*A.size, max_age = (1+np.random.normal(0, .09))*A.max_age, 
                                   moving = (1+np.random.normal(0, .09))*A.moving))
            else:
                A.bare -= 1
            A.x = A.x+int(A.moving*np.trunc(np.random.normal(0, 2)))
            A.y = A.y+int(A.moving*np.trunc(np.random.normal(0, 2)))
            A.x = max(A.x, 0)
            A.x = min(A.x, boundary)
            A.y = max(A.y, 0)
            A.y = min(A.y, boundary)
            
    for A in Bs:
        if A.age < 0:
            Bs.remove(A)
        else:
            # A.age -= 1
            if A.feeding <= 0:
                Bs.remove(A)
            else:
                A.feeding -= 1
                if A.bare < 0:
                    if A.feeding > 300:
                        A.feeding -= 150
                        A.bare = A.max_bare
                        Bs.append(predator(x = A.x, y = A.y, max_bare = (1+np.random.normal(0, .09))*A.max_bare, 
                                           size = (1+np.random.normal(0, .09))*A.size, max_age = (1+np.random.normal(0, .09))*A.max_age, 
                                           moving = (1+np.random.normal(0, .09))*A.moving))
                else:
                    A.bare -= 1
                if A.feeding > 250:
                    A.x = A.x+int(np.trunc(A.moving*np.random.normal(0, 2)))
                    A.y = A.y+int(np.trunc(A.moving*np.random.normal(0, 2)))
                else:
                    long = int(np.trunc(A.moving*np.random.random()*3))
                    if As:
                        near_i = np.argmin([np.linalg.norm([i.x - A.x, i.y - A.y]) for i in As])
                        prey_mean_x = (As[near_i].x - A.x)
                        prey_mean_y = (As[near_i].y - A.y)
                        A.x = A.x + long*((prey_mean_x) / np.linalg.norm([prey_mean_x, prey_mean_y]))
                        A.y = A.y + long*((prey_mean_y) / np.linalg.norm([prey_mean_x, prey_mean_y]))
                A.x = max(A.x, 0)
                A.x = min(A.x, boundary)
                A.y = max(A.y, 0)
                A.y = min(A.y, boundary)
        for prey in As:
            if (np.abs(A.x - prey.x) < (A.size + prey.size)) and (np.abs(A.y - prey.y) < (A.size + prey.size)) and (A.feeding < 500):
                A.feeding += 10
                As.remove(prey)
                
    if t % 1 == 0:
        ax.clear()
        s = ((ax.get_window_extent().width  / (boundary+1.) * 72./fig.dpi) ** 2)
        ax.set_title('Time : {:d}, Preys : {:d}, Predators : {:d}'.format(t, len(As), len(Bs)))
        ax.scatter([i.x for i in As], [i.y for i in As], c = 'b', s = s*np.array([4*i.size**2 for i in As]), marker = 's', edgecolors='k', linewidth = 1)
        ax.scatter([i.x for i in Bs], [i.y for i in Bs], c = 'r', s = s*np.array([4*i.size**2 for i in Bs]), marker = 's', edgecolors='k', linewidth = 1)
        for i in Bs:
            ax.text(i.x, i.y, '$\\bf {:d}$'.format(i.feeding), color='w', ha='center', va='center')
            if As:
                near_i = np.argmin([np.linalg.norm([j.x - i.x, j.y - i.y]) for j in As])
                ax.scatter(As[near_i].x, As[near_i].y, c = 'k', s = s*5**2, marker = 's')
        ax.set_xlim([0, boundary])
        ax.set_ylim([0, boundary])
        ax.set_xticks([0, boundary]), ax.set_yticks([0, boundary])
        plt.pause(.1)
    t += 1
    if not Bs:
        break
    if not As:
        break

KeyboardInterrupt: 

In [ ]:
plt.close()